#### Multi-Agent agent supervisor 
We can use an LLM to orchestrate the different agents.

In [1]:
### Create the tools
from typing import Annotated, List, Tuple, Union
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.tools.python.tool import PythonREPLTool

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
tavily_tool = TavilySearchResults(max_results=2)
python_repl_tool = PythonREPLTool()

In [4]:
### Create helper utility to create agents
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system",  system_prompt), 
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ]
    )
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [5]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

##### Create Agent Supervisor
It will use function calling to choos the next work node or finish processing

In [15]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatMessagePromptTemplate, MessagesPlaceholder


members = ["Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

options = ["FINISH"] + members

function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}


prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    (
        "system",
        "Given the conversation above, who should act next? Or should we FINISH? select on of {options}"
    )
]).partial(options=str(options), members=",".join(members))

llm = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")

supervisor_llm = llm.bind_functions(functions=[function_def], function_call="route")
supervisor_chain = (
    prompt | supervisor_llm
)

In [25]:
supervisor_chain.invoke({"messages": [
            HumanMessage(content="Code hello world and print it to the terminal")
        ]})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8o6zld2oefflout1fa2gqzlz', 'function': {'arguments': '{"next":"Coder"}', 'name': 'route'}, 'type': 'function'}]})

##### Construct the Graph
We are ready to start building the graph. Below, defin eteh state and wordr nodes using thefunction we just defined. 

In [26]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

In [27]:
research_agent = create_agent(llm, [tavily_tool], "You are a web researcher")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")


code_agent = create_agent(llm, [python_repl_tool], "You may generate safe python  code to anlayze data and generate chart using matplotlib")
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")


In [28]:
workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)


for member in members:
    workflow.add_edge(member, "supervisor")
conditional_map = {k:k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["tool_calls"][0]["function"]["arguments"]["next"], conditional_map)
workflow.set_entry_point("supervisor")

#### Invoke the team
With the graph created, we can now invoke it and see how it performs!

In [29]:
graph = workflow.compile()

In [33]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Code hello world and print it to the terminal")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

InvalidUpdateError: Invalid state update, expected dict with one or more of ['messages', 'next'], got content='' additional_kwargs={'tool_calls': [{'id': 'call_yenf56k2vey7q281ggjw48gq', 'function': {'arguments': '{"next":"Coder"}', 'name': 'route'}, 'type': 'function'}]}